In [16]:
from rasa.nlu import train
from rasa.nlu.classifiers import LABEL_RANKING_LENGTH
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.constants import TEXT, SPARSE_FEATURE_NAMES, DENSE_FEATURE_NAMES, INTENT
from rasa.utils.tensorflow.constants import (
    LOSS_TYPE,
    RANDOM_SEED,
    RANKING_LENGTH,
    EPOCHS,
    MASKED_LM,
    TENSORBOARD_LOG_LEVEL,
    TENSORBOARD_LOG_DIR,
    EVAL_NUM_EPOCHS,
    EVAL_NUM_EXAMPLES,
    BILOU_FLAG,
)
from rasa.nlu.classifiers.diet_classifier import DIETClassifier
from rasa.nlu.model import Interpreter
from rasa.nlu.training_data import Message
from rasa.utils import train_utils
from rasa.nlu import components, utils

def as_pipeline(*components):
    return [{"name": c} for c in components]

# classifier_params={RANDOM_SEED: 1, EPOCHS: 1, BILOU_FLAG: False}
classifier_params={RANDOM_SEED: 1, EPOCHS: 100, BILOU_FLAG: False}
pipeline = as_pipeline(
    "WhitespaceTokenizer", "CountVectorsFeaturizer", "DIETClassifier",
    "EntitySynonymMapper"
)
assert pipeline[2]["name"] == "DIETClassifier"
pipeline[2].update(classifier_params)

_config = RasaNLUModelConfig({"pipeline": pipeline, "language": "en"})
tmpdir='out'
component_builder = components.ComponentBuilder()
(trainer, trained, persisted_path) = await train(
    _config,
    path=tmpdir,
    data="demo-rasa-composite-entities.md",
    component_builder=component_builder,
)

Epochs: 100%|██████████| 100/100 [00:28<00:00,  3.54it/s, t_loss=1.535, i_loss=0.217, entity_loss=0.277, group_loss=0.258, role_loss=0.108, i_acc=1.000, entity_f1=0.751, group_f1=0.740, role_f1=0.647]


In [7]:
loaded = Interpreter.load(persisted_path, component_builder)

assert loaded.pipeline
text = "I am looking for an italian restaurant"
assert loaded.parse(text) == trained.parse(text)

Instructions for updating:
Use tf.identity instead.


In [11]:
# loaded.parse(text)
trained.parse("I am looking for an italian restaurant")

{'intent': {'name': 'affirm', 'confidence': 0.30026865005493164},
 'entities': [],
 'intent_ranking': [{'name': 'affirm', 'confidence': 0.30026865005493164},
  {'name': 'order_pizza', 'confidence': 0.29987359046936035},
  {'name': 'book_flight', 'confidence': 0.16730810701847076},
  {'name': 'chitchat', 'confidence': 0.10088472813367844},
  {'name': 'greet', 'confidence': 0.08982272446155548},
  {'name': 'goodbye', 'confidence': 0.04184221476316452}],
 'text': 'I am looking for an italian restaurant'}

In [17]:
trained.parse("Book a flight from Berlin to SF")

{'intent': {'name': 'book_flight', 'confidence': 0.9999995231628418},
 'entities': [{'entity': 'location',
   'start': 19,
   'end': 25,
   'role': 'to',
   'value': 'Berlin',
   'extractor': 'DIETClassifier'},
  {'entity': 'location',
   'start': 29,
   'end': 31,
   'role': 'from',
   'value': 'San Fransisco',
   'extractor': 'DIETClassifier',
   'processors': ['EntitySynonymMapper']}],
 'intent_ranking': [{'name': 'book_flight', 'confidence': 0.9999995231628418},
  {'name': 'chitchat', 'confidence': 3.4653672287277004e-07},
  {'name': 'affirm', 'confidence': 9.466146622116867e-08},
  {'name': 'order_pizza', 'confidence': 5.335387243121659e-08},
  {'name': 'greet', 'confidence': 4.062503577983989e-08},
  {'name': 'goodbye', 'confidence': 9.429986791431588e-10}],
 'text': 'Book a flight from Berlin to SF'}

In [19]:
from rasa.nlu import training_data
td = training_data.load_data("demo-rasa-composite-entities.md")
td.entity_synonyms, td.intents

({'SF': 'San Fransisco'},
 {'affirm', 'book_flight', 'chitchat', 'goodbye', 'greet', 'order_pizza'})

In [20]:
td.examples_per_entity["entity 'location'"]

8

In [21]:
from rasa.nlu.convert import convert_training_data
from rasa.nlu.training_data.util import get_file_format
get_file_format("demo-rasa-composite-entities.md")

'md'

In [22]:
data_file="demo-rasa-composite-entities.md"
out_path = tmpdir+'/rasa_nlu_data.json'
convert_training_data(data_file, out_path, output_format='json', language=None)